# Fault Diagnosis Toolbox in Python
## - Basic model definition and class methods

Erik Frisk<br>
<frisk@isy.liu.se><br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

## Import modules

In [1]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import sys
import numpy as np
import sympy as sym
import scipy.sparse as sp

Set paths and import toolbox + misc functionality

In [2]:
new_paths = ['../Models/', '../Misc/', 'faultdiagnosistoolbox/build/lib.macosx-10.6-x86_64-2.7/']
[sys.path.append(d) for d in new_paths if not d in sys.path];
from misc import *
import faultdiagnosistoolbox as fdt

## Define a diagnosis models -- structural Three Tank model

In [3]:
#from ThreeTank_model import model
from ThreeTank_model_sym import model
model.Lint()

Model: Three Tank System

  Type:Symbolic, dynamic

  Variables and equations
    10 unknown variables
    3 known variables
    6 fault variables
    12 equations, including 3 differential constraints

  Degree of redundancy: 2
  Degree of redundancy of MTES set: 1


  Model validation finished with 0 errors and 0 warnings.


In [ ]:
np.where(np.any(model.Z.toarray()[:,[0,2]],axis=1))[0]

In [ ]:
def MeasurementEquations(self, m):
    mIdx = np.array([self.z.index(zi) for zi in m if zi in self.z])
    return np.where(np.any(model.Z.toarray()[:,mIdx],axis=1))[0]
    
list(model.syme[MeasurementEquations(model,['y1','y3'])])

Plot model structure

In [ ]:
plt.figure(10)
model.PlotModel(verbose=True)
plt.show()

Plot the Dulmage-Mendelsohn decomposition with equivalence class decomposition

In [ ]:
plt.figure(11)
model.PlotDM(eqclass=True,fault=True)
plt.show()

## Apply some class methods

Are the faults detectable?

In [ ]:
model.DetectabilityAnalysis()

Some redundancy properties of the model

In [ ]:
print "Redundancy = %d" % model.Redundancy()
print "MTES Redundancy = %d" % model.MTESRedundancy()

What about isolability?

In [ ]:
plt.figure(12)
plt.subplot(1,3,1)
model.IsolabilityAnalysis(permute=True,plot=True)
plt.title('Mixed')

plt.subplot(1,3,2)
im = model.IsolabilityAnalysis(permute=True,plot=True, causality='int')
plt.title('Integral')

plt.subplot(1,3,3)
im = model.IsolabilityAnalysis(permute=True,plot=True, causality='der')
plt.title('Integral')
plt.show()

Find the set of MSO sets

In [ ]:
print "Searching for MSO sets..."
msos=model.MSO()
print "Found %d mso sets" % len(msos)
print msos

Determine which MSO sets that are low index

In [ ]:
li_msos = map(lambda m: model.IsLowIndex(eq=m), msos)
print li_msos

Determine the fault signature and isolability matrix for MSO sets

In [ ]:
plt.figure(13)
plt.subplot(1,2,1)
model.FSM(msos,plot=True)
plt.title('FSM')

plt.subplot(1,2,2)
model.IsolabilityAnalysisArrs(msos, plot=True, permute=True)
plt.title('Isolability matrix')
plt.show()

Find an integral causality matching, if one exists

In [ ]:
mso = msos[2]
rIdx = model.MSOCausalitySweep(mso,causality='int') # Boolean index array to integral causality redundant equation
if np.any(rIdx):
    red = mso[rIdx][0] # Take the first one
    m0 = [e for e in mso if e!=red]
    Gamma = model.Matching(m0)
    print "Found integral causality matching"
else:
    print "No integral causality matching exists"

In [ ]:
plt.figure(14)
model.PlotMatching(Gamma)
plt.show()

In [ ]:
model_ld = model.copy()
model_ld.LumpDynamics()
plt.figure(20)
model_ld.PlotModel()
model_ld.Lint()